# Reconocimiento de Patrones
[Curso de Reconocimiento de Patrones]

Departamento Electrónica

Universidad Popular del Cesar

José Ramón Iglesias



# IMPORTANTE:

En este código por favor:

*   No agregar celdas nuevas
*   No agregar librerías nuevas
*   No usar otro nombre de variables
*   No cambiar el clasificador ni el número de vecinos del KNN
*   No hacer gráficas


<font color='red'>Esta tarea será corregida automáticamente, y si no se cumple alguna de las condiciones anteriores, es posible que la tarea no se corrija correctamente.</font>

Si quieres hacer modificaciones y probar estrategias distintas, te invito a hacerlo en otro código (no en éste). <font color='red'>Este código debe modificarse sólo en los lugares indicados.</font>

<font color='orange'> Evaluación:</font>
 Este ejercicio tiene n=15 ítems a evaluar, denotados como Q00, Q01, ... Qn a lo largo del notebook como `<Evaluación Qi>`. En tu evaluación (que recibirás en los próximos días) se indicará cómo han sido cada uno de estos n items. Tu nota en este ejercicio será el número de ítems correctamente hechos dividido por n multiplicado por 100 (en escala de 0 a 100%).


# Ejercicio 05: Detección de paredes rayadas usando LBP y HoG
----
ENTREGA: viernes 30/septiembre/2022 a la 1:00 pm (se debe mostrar al profesor funcionando este archivo ipynb, <font color='red'>el nombre del archivo debe coincidir con tu número de alumno/a)</font>.

----
En este ejercicio se realizará la clasificación entre dos clases de "parches" de pared de 64x64 pixeles (en escala de grises):

* Clase 0: No-Rayas (contiene parches de pared sin rayas). [Ver parches no rayados](https://www.dropbox.com/s/2e0mxapuc3tno5j/patches_0.png?dl=0)

* Clase 1: Rayas (contiene parches de pared con pintas, grafitis, rayados, etc.). [Ver parches rayados](https://www.dropbox.com/s/694u3ttbjmxx2lq/patches_1.png?dl=0)




La base de datos contiene 1200 imágenes de 64x64 pixeles (600 imágenes por clase).

**ADVERTENCIA:** Este ejercicio tiene fines pedagógicos sólamente, con la idea de que la solución a este problema pueda ejecutarse en un par de minutos. Un buen detector de rayas sigue esta idea pero con miles de imágenes a color por clase.

A continuación, en cada PREGUNTA deberás ingresar alguna(s) líneas de código, o completar donde aparezca '<= COMPLETAR AQUI".




# PREGUNTA 0: Cambiar nombre de archivo

Cambia el nombre del archivo de este código de tal forma que sea tu número de alumno/a. Por ejemplo, si tu número de alumno/a es '1234567J' entonces este Colab Notebook debe llamarse '1234567J.ipynb'. Para hacerlo debes hacer un click con el mouse sobre el nombre del archivo (arriba a la izquierda) y editar el nombre del archivo.

In [ ]:
Nombre_de_archivo_cambiado =  1  # <== COMPLETAR AQUI ESCRIBIENDO EL NÚMERO 1 CUANDO HAYAS CAMBIADO EL NOMBRE

# PREGUNTA 1: Datos personales

Ingresa tus datos para poder evaluarte, 
<font color='red'>si tus datos son incorrectos, este ejercicio no podrá ser corregido y por lo tanto no tendrás nota en esta actividad.</font>

In [ ]:
NumeroID         = '12345678J'   # <= COMPLETAR AQUI NUMERO DE ALUMNO/A ENTRE APOSTROFES
PrimerNombre     = 'José Ramón'     # <= COMPLETAR AQUI TU PRIMER NOMBRE
PrimerApellido   = 'Iglesias'        # <= COMPLETAR AQUI TU PRIMER APELLIDO
Correo           = 'joseiglesias@unicesar.edu.co'   # <= COMPLETAR AQUI TU CORREO ELECTRONICO
Datos_ingresados = 1             # <= COMPLETAR AQUI ESCRIBIENDO 1 CUANDO HAYAS INGRESADO TUS DATOS

stid             = PrimerNombre+' '+PrimerApellido+' (ID:'+NumeroID+', email:'+Correo+'):'
print('Registro: '+stid)

Registro: Domingo Mery (ID:12345678J, email:dmery@ing.puc.cl):


# Setup inicial

Liberías necesarias para que funcione el algoritmo.

## Instalación de PyBalu

[PyBalu](https://github.com/mbucchi/pybalu) es una librería creada para extraer características. 

In [ ]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

from IPython.display import clear_output
!pip install scipy==1.2
!pip3 install pybalu==0.2.5
clear_output()
print('Librería PyBalu instalada.')

Librería PyBalu instalada.


## Setup de librerías

In [ ]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

import numpy as np
import matplotlib.pyplot as plt
import os.path
from   sklearn.metrics import confusion_matrix, accuracy_score
from   sklearn.neighbors import KNeighborsClassifier
from   tqdm.auto import tqdm
from   pybalu.feature_extraction import lbp_features, hog_features
from   pybalu.feature_transformation import normalize

print('Librerías cargadas.')


Librerías cargadas.


## Funciones necesarias

In [ ]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

def num2fixstr(x,d):
  # example num2fixstr(2,5) returns '00002'
  # example num2fixstr(19,3) returns '019'
  st = '%0*d' % (d,x)
  return st

def ImageLoad(prefix,num_char,num_img,echo='off'):
  st   = prefix + num2fixstr(num_char,2) + '_' + num2fixstr(num_img,3) + '.png'
  if echo == 'on':
    print('loading image '+st+'...')
  img    = plt.imread(st)
  return img

def MatrixStats(X):
  p = np.zeros((6),'int')
  n = len(X.shape)
  if n>0:
    p[0] = X.shape[0]
    if n==2:
      p[1] = X.shape[1]
  p[2] = X.min()
  p[3] = X.max()
  p[4] = X.mean()
  p[5] = X.std()
  return p
  
def CheckMatrix(st,X,ps):
  p = MatrixStats(X)
  d = p-ps
  t = not(np.sum(d)==0)
  if t:
    print(st + ': Revisar, esta matriz presenta errores...')
  else:
    print(st + ': ok')
  return t

def CheckPrint(Y,st=True):
    
  if np.sum(Y)>0:
    if st:
      print('Revisar, hay error en al menos uno de estos cálculos, no continuar con las siguientes celdas hasta no corregir este error.')
    else:
      print('Revisar, hay error en al menos uno de los datos personales (no coinciden con los ingresados en la PREGUNTA 1),')
      print('NO SUBIR TAREA ANTES DE CORREGIR ESTE ERROR.')
      print('(tu ejercicio corre el riesgo de no ser corregido)')
  else:
    if st:
      print('Felicitaciones! Los cálculos han sido realizados correctamente.')
    else:
      print('Felicitaciones! ahora puedes mostrarle este archivo al profesor.')


def CheckFile(wclass):
  st = 'W_'+num2fixstr(wclass,2)
  sfile = st+'_600.png'
  x = os.path.isfile('rayas/'+sfile)
  if x:
      print('Imágenes '+st+'*.png: OK')
  else:
      print('Carga de imágenes '+st+' presentan errores.')
  return not(x)

chk = [[1200,59,0,1524,69,136],[1200,9,0,98,26,19],[1200,0,0,100,50,50],[ 900,68,0,1524,60,129],[ 300,68,0,1432,60,129],[900,0,0,100,50,50],[300,0,0,100,50,50],[0,0,91,91,91,0],[2,2,13,137,75,61]]

# Separación entre training y testing
def SplitTrainTest(X,y,n):

  K      = np.max(y)+1              # número de clases
  N      = np.int(X.shape[0]/K)     # numeros de muestras por clase
  Ntrain = n*K                      # número de muestras para el training
  Ntest  = K*N-Ntrain               # número de muestras para el testing
  M      = X.shape[1]               # número de características por muestra
  Xtrain = np.zeros((Ntrain,M))     # subset de training
  ytrain = np.zeros((Ntrain),'int') # ground truth del training         
  Xtest  = np.zeros((Ntest,M))      # subset de testing
  ytest  = np.zeros((Ntest),'int')  # ground truth del testing  

  # contadores
  itrain = 0
  itest  = 0
  t      = 0

  for j in range(K):     # para cada clase
    for i in range(N):   # para cada imagen de la clase
      if i<n: # training
        Xtrain[itrain,:] = X[t,:]
        ytrain[itrain] = y[t]
        itrain = itrain+1
      else:  # testing
        Xtest[itest,:] = X[t,:]
        ytest[itest] = y[t]
        itest = itest+1
      t = t+1
  
  return Xtrain,ytrain,Xtest,ytest

# Clasificación usando KNN con 1 vecino
def ClassifierKNN(Xtrain,ytrain,Xtest,ytest):
  Xtrain, a, b = normalize(Xtrain)
  Xtest        = Xtest * a + b
  knn = KNeighborsClassifier(n_neighbors=1)
  knn.fit(Xtrain, ytrain)
  ypred        = knn.predict(Xtest)
  acc          = accuracy_score(ytest,ypred)
  print('Entrenando con '+str(Xtrain.shape[0])+' muestras y probando con '+str(Xtest.shape[0])+' muestras')
  print('Testing Accuracy = '+str(acc*100)+'%')
  C = confusion_matrix(ytest,ypred)
  print('Matriz de Confusión:')
  print(C)
  return acc,C

print('Funciones necesarias ejecutadas.')

Funciones necesarias ejecutadas.


# Pregunta 2: Carga de base de datos

La base de datos consiste en 2 clases con 600 imágenes por clase. Se almacenan en la carpeta `rayas` con el formato `W_xx_nnn.png`, donde `xx` es el ID de la clase (01 para no-rayas, 02 para raya) y `nnn` es el número de la foto de la clase (001, 002, ... 600). Las fotos son de 64x64 pixeles en escala de grises.


Carga la base de datos, descargando el archivo rayas.zip que se encuentra en este link: `https://www.dropbox.com/s/lf8n9lyb5k2eboq/rayas.zip?dl=0`. Una vez descargado el archivo zip, debes descomprimirlo.

AYUDA: usar comando `!wget <url archivo zip>` para descargar el archivo, y el comando `! unzip <archivo zip>` para descomprimirlo.



In [ ]:
# Carga de base de datos

# COMPLETAR AQUI DOS LINEAS DE CODIGO
# Linea 1: comando para bajar archivo zip desde dropbox
# Linea 2: comando para descomprimir el archivo zip bajado en la linea 1
# Al final de la ejecucion debe haberse creado la carpeta rayas con 1200 imágenes

!wget https://www.dropbox.com/s/lf8n9lyb5k2eboq/rayas.zip?dl=0 # <- COMPLETAR AQUI LINEA 1
!unzip rayas.zip # <- COMPLETAR AQUI LINEA 2


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA CARGA DE LA BASE DE DATOS
#

T = np.ones((15),'int')
T[0] = CheckFile(1) # <Evaluación Q00>
T[1] = CheckFile(2) # <Evaluación Q01>
CheckPrint(T[0:2])

Imágenes W_01*.png: OK
Imágenes W_02*.png: OK
Felicitaciones! Los cálculos han sido realizados correctamente.


# PREGUNTA 3: Extracción de características

En esta pregunta se debe realizar la extracción de características de la imagen `img`. Las características a extraer son las basadas en LBP y Hog y se extraen usando los siguientes comandos de la librería PyBalu:

## 1) Cómo extraer características LBP clásico:

Con el comando

`f  = lbp_features(img, hdiv=a, vdiv=b, mapping='nri_uniform')`

se divide la imagen `img` en `a x b` particiones y en cada una de ellas se extrae el descriptor LBP (vector de 59 elementos). En este caso `f` es un vector de `59*a*b` elementos. En este ejercicio trabajaremos con `a = b = 1`, es decir sólo una partición por imagen.

## 2) Cómo extraer características de HoG:

Con el comando

`f = hog_features(img, v_windows=a, h_windows=b, n_bins=q)`

se divide la imagen `img` en `a x b` particiones y en cada una de ellas se extrae el descriptor HoG (vector de `q` elementos, ya que el histograma es de `q` bins). En este caso `f` es un vector de `q*a*b` elementos. En este ejercicio trabajaremos con `a = b = 1`, es decir sólo una partición por imagen, y usaremos `q = 9` bins en el histograma.

<font color='red'> Para escribir este código reutilice el código del Ejercicio 04 (celda dedicada a extracción de características) teniendo en cuenta las indicaciones 1) y 2). Este código tiene alrededor de 20-30 líneas.

In [ ]:
#print('Ejecución de '+stid) # <== NO MODIFICAR ESTA LINEA

# Extracción de Características

# En esta celda se debe: 
#  - almacenar en la variable Xlbp la matriz de las características LBP de las 1200 imágenes
#  - almacenar en la variable Xhog la matriz de las características HOG de las 1200 imágenes
#  - almacenar en la variable y el vector de la clase a que pertenece cada una de las 1200 imágenes

# Al final de la ejecución de este código deben haber sido creadas:
# Matriz Xlbp
# Matriz Xhog 
# Vector y


K    = 2     # <= COMPLETAR AQUI EL NUMERO DE CLASES
N    = 600   # <= COMPLETAR AQUI EL NUMERO DE IMAGENES POR CLASE
Mlbp = 59    # <= COMPLETAR AQUI EL NUMERO DE ELEMENTOS DEL VECTOR LBP
Mhog = 9     # <= COMPLETAR AQUI EL NUMERO DE ELEMENTOS DEL VECTOR DE HOG

Xlbp  = np.zeros((K*N,Mlbp))    # K*N muestras (filas), y Mlbp características (columnas)
Xhog  = np.zeros((K*N,Mhog))    # K*N muestras (filas), y Mhog características (columnas)

y     = np.zeros((K*N),'int')   # ground truth (clasificacion ideal)

t = 0
print('Cargando imagenes y extrayendo características...')
for j in range(K):            # para cada clase
  print('Extrayendo características: batch '+str(j+1)+'/'+str(K))
  for i in tqdm(range(N)):                # para cada imagen de la clase
    # Lectura de la imagen
    img    = 255*ImageLoad('rayas/W_',j+1,i+1,echo='off')

    # Extracción de características

    # LBP clásico de 1x1 particiones
    Xlbp[t,:] = lbp_features(img, hdiv=1, vdiv=1, mapping='nri_uniform') # <= COMPLETAR AQUI PARA EXTRAER LBP (VER HELP MAS ARRIBA)

    # Haralick con d = 3 pixeles
    Xhog[t,:] = hog_features(img, v_windows=1, h_windows=1, n_bins=9)    # <= COMPLETAR AQUI PARA EXTRAER HOG (VER HELP MAS ARRIBA)



    y[t] = j # la muestra t pertenece a la clase j
    t = t+1


Cargando imagenes y extrayendo características...
Extrayendo características: batch 1/2



Extrayendo características: batch 2/2


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA EXTRACCiÓN DE CARACTERÍSTICAS
#

T[2] = CheckMatrix('Xlbp',Xlbp,chk[0])     # <Evaluación Q02>
T[3] = CheckMatrix('Xhog',Xhog*100,chk[1]) # <Evaluación Q03>
T[4] = CheckMatrix('y   ',y*100,chk[2])       # <Evaluación Q04>
CheckPrint(T[2:5])

Xlbp: ok
Xhog: ok
y   : Revisar, esta matriz presenta errores...
Revisar, hay error en al menos uno de estos cálculos, no continuar con las siguientes celdas hasta no corregir este error.


# Clasificación usando LBP y HoG

En la pregunta anterior, fueron extraídas las características LBP y HoG para todas las imágenes de la base de datos. No olvidar que son 600 imágenes por clase, y que son 2 clases. En este Ejercicio, el clasificador será  un clasificador KNN de 1 vecino que será entrenado (training) con las primeras 450 muestras de cada clase, y probado (testing) con las 150 restantes por clase.



# Pregunta 4: Definicion de características a utilizar

Cargar en `X` la concatenación de `Xlbp` y `Xhog`. Es decir, si `Xlbp` tiene `P x Mlbp` elementos, y `Xhog` tiene `P x Mhog` elementos, entonces `X` debe tener `P x (Mlbp+Mhog)` elementos.

Ayuda: use función `np.concatenate()`


In [ ]:
# Al final de esta ejecución debe haber sido creada la matriz X

X = np.concatenate((Xlbp,Xhog),axis=1)  # <= COMPLETAR AQUI LA LÍNEA DE CÓDIGO QUE REALIZA LA CONCATENACIÓN DE LBP Y HOG (NO CAMBIAR EL ORDEN)


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA DEFINICION DE X
#

T[5] = CheckMatrix('X',X[:,0:59],chk[0])    # <Evaluación Q05>
T[6] = CheckMatrix('X',X[:,59:]*100,chk[1]) # <Evaluación Q06>
CheckPrint(T[5:7])

X: ok
X: ok
Felicitaciones! Los cálculos han sido realizados correctamente.


# PREGUNTA 5: Separación Training / Testing

El conjunto de training debe contener las primeras 450 muestras por classe. El conjunto de testing debe contener las restantes 150 muestras por clase.

Separar `X,y` en `Xtrain,ytrain`, `Xtest,ytest`

Ayuda: usar funcion `SplitTrainTest()` definida más arriba en la seeción "Funciones Necesarias".

In [ ]:
# Al final de esta ejecución debe haber sido creada la matrices Xtrain, Xtest y los vectores ytest , ytrain

Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,450)  # <= COMPLETAR AQUI LA LÍNEA DE CÓDIGO QUE REALIZA LA SPERACIÓN TRAINING/TESTING


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA SEPARACION TRAINING/TESTING
#

T[7]  = CheckMatrix('Xtrain',Xtrain,chk[3])     # <Evaluación Q07>
T[8]  = CheckMatrix('Xtest ',Xtest ,chk[4])     # <Evaluación Q08>
T[9]  = CheckMatrix('ytarin',ytrain*100,chk[5]) # <Evaluación Q09>
T[10] = CheckMatrix('ytest ',ytest*100 ,chk[6]) # <Evaluación Q10>

CheckPrint(T[7:11])


Xtrain: ok
Xtest : ok
ytarin: ok
ytest : ok
Felicitaciones! Los cálculos han sido realizados correctamente.


# PREGUNTA 6: Clasificación

Diseñar el clasificador usando `Xtrain,ytrain`, pruebe el clasificador en `Xtest`, y compare la clasifición con `ytest`.

Ayuda: usar funcion `ClassifierKNN()` definida más arriba en la seeción "Funciones Necesarias".


In [ ]:
# Al final de esta ejecución debe haber sido creada la variable acc y la matriz de confusion C

acc,C = ClassifierKNN(Xtrain,ytrain,Xtest,ytest)  # <= COMPLETAR AQUI LA LÍNEA DE CÓDIGO QUE REALIZA LA CLASIFICACIÓN


Entrenando con 900 muestras y probando con 300 muestras
Testing Accuracy = 91.0%
Matriz de Confusión:
[[136  14]
 [ 13 137]]


In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA CLASIFICACIÓN CON LBP Y HOG
#

T[11]  = CheckMatrix('acc',acc*100,chk[7]) # <Evaluación Q11>
T[12] = CheckMatrix('C  ',C ,chk[8])         # <Evaluación Q12>
CheckPrint(T[11:13])


acc: ok
C  : ok
Felicitaciones! Los cálculos han sido realizados correctamente.


# PREGUNTA 7: Verificar datos personales

<font color='red'> **ADVERTENCIA:** En este ejercicio en necesario que revises bien tus datos personales. Si no son correctos, no tendrás nota en este ejercicio.

In [ ]:
# POR FAVOR VUELVE A DIGITAR ESTOS DATOS PARA VERIFICAR QUE NO HAYA ERRORES DE TIPEO
NumeroID_2       = '12345678J' # <= COMPLETAR AQUI NUMERO DE ALUMNO/A ENTRE APOSTROFES
Correo_2         = 'joseiglesias@unicesar.edu.co' # <= COMPLETAR AQUI TU CORRECO ELECTRONICO

In [ ]:
#
# EJECUTAR ESTA CELDA PARA VERIFICAR SI LOS DATOS COINCIDEN CON LOS YA INGRESADOS
#

T[13] = NumeroID != NumeroID_2  # <Evaluación Q13>
T[14] = Correo   != Correo_2    # <Evaluación Q14>
CheckPrint(T[13:15],0)

Felicitaciones! ahora puedes subir este archivo a Google Classroom.


# PASO FINAL: Subir tarea

<font color='red'> **ADVERTENCIA FINAL:** Debes verificar que todas las celdas de este código hayan sido ejecutadas sin error. Si durante la evaluación automática tu código "se cae" en una celda, las siguientes celdas no serán ejecutadas (ni evaludas). 

<font color='orange'> Este archivo, que debe llamarse [tu NumeroID].ipynb, debe mostrarse al pofesor en la actividad Ejercicio 05 que se entrega a más tardar el viernes 30/septiembre/2022 a la 1:00 pm.
